# Problem Statement

We want to build an online shopping cart system that allows users to add products to their cart, calculate the total cost, apply discounts, and generate an invoice. The system should include the following functionalities:

- Adding products to the cart
- Removing products from the cart
- Calculating the total cost
- Applying discounts based on user type
- Generating an invoice

### 1. Create the Product class

We create a basic `Product` class with attributes for the product name and price.

In [1]:
class Product:
    def __init__(self, name, price):
        self.name = name
        self.price = price

### 2.  Implement the User class

In this step, we create a `User` class with attributes for the user's name and whether they are a premium member. We then modify the `calculate_total_cost` method in the `ShoppingCart` class to apply a `10%` discount for premium users.

In [2]:
class User:
    users = []  # Class variable to store all created users

    def __init__(self, name):
        self.name = self.validate_name(name)
        self.__is_premium = False
        self.__is_admin = False
        self.cart = ShoppingCart()
        User.users.append(self)  # Add the user to the list of users
        if self.name == 'admin':
            self.__is_premium = True
            self.__is_admin = True

    @staticmethod
    def validate_name(name):
        if any(user.name == name for user in User.users):
            print(f"User with the name '{name}' already exists. Please choose a different name.")
            unique_name = input("Please enter a unique name: ")
            return User.validate_name(unique_name)  # Recursively validate the unique name
        return name
    
    @property
    def is_admin(self):
        return self.__is_admin
    
    @property
    def is_premium(self):
        return self.__is_premium

    def make_admin(self, admin_user):
        if admin_user.is_admin:
            self.__is_admin = True
            
    def make_premium(self, admin_user):
        if admin_user.is_admin:
            self.__is_premium = True

### 3. Create the ShoppingCart class

In this step, we create a `ShoppingCart` class with methods for adding and removing products from the cart, as well as calculating the total cost of the items in the cart.

In [3]:
def discount_10_percent(func):
    def wrapper(self):
        total_cost = func(self)
        discounted_cost = total_cost * 0.9
        return discounted_cost
    return wrapper

Here, we defined a decorator `discount_10_percent` that applies a `10%` discount to the total cost. We then apply this decorator to the `calculate_total_cost` method in the `ShoppingCart` class.

In [4]:
class ShoppingCart:
    def __init__(self):
        self.products = []

    def add_product(self, product):
        self.products.append(product)

    def remove_product(self, product):
        self.products.remove(product)

    def calculate_total_cost(self):
        total_cost = sum(product.price for product in self.products)
        return total_cost
    
    @discount_10_percent
    def calculate_discounted_cost(self):
        total_cost = sum(product.price for product in self.products)
        return total_cost
    
    def generate_invoice(self, user):
        invoice = f"Invoice for {user.name}:\n"
        invoice += "=======================\n"
        for product in self.products:
            invoice += f"{product.name}: ${product.price}\n"
        total_cost = self.calculate_total_cost()
        if user.is_premium:
            invoice += "-----------------------\n"
            invoice += f"Sub-Total: ${total_cost}\n"
            final_cost = self.calculate_discounted_cost()
            invoice += f"Discount (10%): ${total_cost - final_cost}\n"
            total_cost = final_cost
        invoice += "-----------------------\n"
        invoice += f"Total: ${total_cost}"
        return invoice

    def get_products(self):
        yield from self.products

### 4. Testing the functionality

Now that we have implemented the necessary classes and methods, let's test our online shopping cart system:

In [5]:
# Create some products
product1 = Product("Shirt", 20)
product2 = Product("Pants", 30)
product3 = Product("Shoes", 50)

# Create a shopping cart
cart = ShoppingCart()

# Add products to the cart
cart.add_product(product1)
cart.add_product(product2)

# Use the generator to iterate over the products
for product in cart.get_products():
    print(f"Product: {product.name}, Price: {product.price}")

Product: Shirt, Price: 20
Product: Pants, Price: 30


### 5. Create an admin user

In [6]:
admin = User('admin')

In [7]:
print(admin.is_admin)
print(admin.is_premium)

True
True


### 6. Generating Invoice for a regular user

In [8]:
# Create a user
user = User("rita")

# Add product to cart
user.cart.add_product(product1)
user.cart.add_product(product2)

# Generate and print the invoice
invoice = user.cart.generate_invoice(user)
print(invoice)

Invoice for rita:
Shirt: $20
Pants: $30
-----------------------
Total: $50


### 7. Generating Invoice for a premium user

In [9]:
# Create a user
user = User("john")

# Make this user a premium user
user.make_premium(admin)

# Add product to cart
user.cart.add_product(product1)
user.cart.add_product(product2)
user.cart.add_product(product3)

# Generate and print the invoice
invoice = user.cart.generate_invoice(user)
print(invoice)

Invoice for john:
Shirt: $20
Pants: $30
Shoes: $50
-----------------------
Sub-Total: $100
Discount (10%): $10.0
-----------------------
Total: $90.0
